In [56]:
from datetime import datetime

In [57]:
# Clase vértice - Usuários
class User:

  def __init__(self, id:int, user:str, email:str):

    """
    user: Usuário da rede
    Id: Id do usuário
    email: email do usuário
    """
    self.id = id
    self.user = user
    self.email = email
    self.hobbies = [] # Tags com interesses.

    
  def __str__(self): # Vai printar o objeto usuário como string
    return str(self.user)

  # Adiciona hobbies dos usuários  
  def addHobbie(self, hobbie):
    self.hobbies.append(hobbie)

In [58]:
# Classe Aresta - Conexões
class Conection:

  def __init__(self, user_1: str, user_2: str, n_interacoes: int):

    """
    User_1: Usuário seguidor
    User_2: Usuário que é seguido por User_1
    n_interacoes: Quantas vezes o usuário A curtiu, comentou ou compartilhou postagens do usuário B.
    """

    self.user_1 = user_1
    self.user_2 = user_2
    self.n_interacoes = n_interacoes

In [59]:
# Classe post
class Post:
    def __init__(self, id:int, author:User, content: str):
        self.id = id
        self.author = author
        self.content = content
        self.date = datetime.now()
        self.likes = set() # Usei o set para evitar duplicatas.

    def like(self, user: User):
        self.likes.append(user)

    def __str__(self):
        return f"Post {self.id} por {self.author.user}: {self.content}."

In [60]:
# Grafo - Unipoli
class UniPoli:

  def __init__(self):
    self.users = {} # Dicionário para armazenar usuários
    self.conections = [] # Lista para armazenar conexões
    self.posts = []  # Lista para armazenar post


  # Adiciona usuários
  def addUser(self, id, user, email):

    if user not in self.users:
      self.users[user] = User(id, user, email)


  # Adiciona conexões - Seguir
  def addConection(self, user_1, user_2, n_interacoes):

    if user_1 and user_2 in self.users:
      a = self.users[user_1]
      b = self.users[user_2]
      conexao = Conection(a, b, n_interacoes)
      self.conections.append(conexao)


  # Exclui usuário
  def delUser(self, user):
    if user in self.users:
      del self.users[user]
      # Apagar as arestas ligadas a user -  filtro
      self.conections = [a for a in self.conections if a.user_1.user != user and a.user_2.user != user]
      print(f'Usuário {user} removido com sucesso.')
      return True
    else:
        print(f'Usuário {user} não encontrado.')
        return False


  # Exclui aresta - Deixar de seguir
  def delConection(self, a, b):
    if a in self.users and b in self.users:
      for aresta in self.conections:
        if aresta.user_1.user == a and aresta.user_2.user == b:
          self.conections.remove(aresta)
          print(f'{a} deixou de seguir {b}.')
          return True
      print(f'{a} não segue {b}.')
    else:
      print(f'{a} or {b} não encontrado.')
    return False


    # Método postar
  def create_post(self, author_name, content: str):
    if author_name in self.users:
        author = self.users[author_name]
        post = Post(len(self.posts), author , content)
        self.posts.append(post)
        return post
    return None # Se autor não existe


  def feed(self, user: str):
    feed = []
    if user in self.users:
        for post in self.posts:
            if (post.author == self.users[user] or # Post do próprio usuário
               post.author in self.conections): # Post de amigos
                    feed.append(post)
        return sorted(feed, key=lambda p: p.date, reverse=True)  # Mais recentes primeiro
      
  # Mostrar grafo
  def display(self):
    print('\n=============================== UniPoli ================================\n')
    print('Usuários: ')
    for user in self.users:
      print(f'{user} ', end=' ')

    print('\n \nConexões: ')
    for user in self.conections:
      print(f'{user.user_1} segue {user.user_2}. {user.user_1} interagiu {user.n_interacoes} vezes com o perfil de {user.user_2}.\n')

    # Printar os posts
    print('\nPosts: ')
    for post in self.posts:
        print(f'Post #{post.id} de {post.author.user}: {post.content}. {post.date}\n')


  # Recomendação para um usuário específico
  def recommend(self, user_ref):
    pass


  # Atualiza o número de interações
  def updateLike(self, user_1, user_2):
    # Procura a aresta existente
    for aresta in self.conections:
        if aresta.user_1.user == user_1 and aresta.user_2.user == user_2:
            aresta.n_interacoes += 1
            return True  # Atualizou com sucesso

    return False  # Usuário(s) inexistente(s)

In [63]:
## Teste
rede = UniPoli()

# Usuários
rede.addUser(0, 'Mikael', 'mikael@gmail.com')
rede.addUser(1, 'Madalena', 'Mad@gmail.com')
rede.addUser(2, 'Ana', 'Ana@gmail.com')
rede.addUser(3, 'Elon Musk', 'El@gmail.com')

# Conexões
rede.addConection('Mikael', 'Madalena', 1)
rede.addConection('Madalena', 'Mikael', 1)
rede.addConection('Madalena', 'Ana', 1)
rede.addConection('Ana', 'Madalena', 1)
rede.addConection('Mikael', 'Elon Musk', 100)

rede.display()

rede.updateLike('Mikael', 'Madalena')
rede.delConection('Mikael', 'Elon Musk')
rede.delConection('Mikael', 'Ana')
rede.delUser('Elon Musk')
rede.create_post('Mikael', 'Eu sou muito esperto!')
rede.create_post('Ana', 'Mikael é um gostoso!')
rede.create_post('Madalena', 'Mikael é super inteligente!')

rede.display()

# Testar o feed
# Feed de Mikael
print("Feed de Mikael:")
for post in rede.feed('Mikael'):
    print(f"- {post}")


=============================== UniPoli ================================

Usuários: 
Mikael  Madalena  Ana  Elon Musk  
 
Conexões: 
Mikael segue Madalena. Mikael interagiu 1 vezes com o perfil de Madalena.

Madalena segue Mikael. Madalena interagiu 1 vezes com o perfil de Mikael.

Madalena segue Ana. Madalena interagiu 1 vezes com o perfil de Ana.

Ana segue Madalena. Ana interagiu 1 vezes com o perfil de Madalena.

Mikael segue Elon Musk. Mikael interagiu 100 vezes com o perfil de Elon Musk.


Posts: 
Mikael deixou de seguir Elon Musk.
Mikael não segue Ana.
Usuário Elon Musk removido com sucesso.

=============================== UniPoli ================================

Usuários: 
Mikael  Madalena  Ana  
 
Conexões: 
Mikael segue Madalena. Mikael interagiu 2 vezes com o perfil de Madalena.

Madalena segue Mikael. Madalena interagiu 1 vezes com o perfil de Mikael.

Madalena segue Ana. Madalena interagiu 1 vezes com o perfil de Ana.

Ana segue Madalena. Ana interagiu 1 vezes com o per